<a href="https://colab.research.google.com/github/ratsgo/nlpbook/blob/master/examples/document_classification/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기
pip 명령어로 의존성 있는 패키지를 설치합니다.

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 778kB 14.3MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 317kB 23.2MB/s 
     |████████████████████████████████| 890kB 39.4MB/s 
     |████████████████████████████████| 1.1MB 38.7MB/s 
     |████████████████████████████████| 3.0MB 32.1MB/s 
     |████████████████████████████████| 276kB 41.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=27d20dd3477e4a04fe1eb7c6e6d4e272c79e39f607f345abf0fd60a291a74b5d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=6023359e7221bfb7bee66894bcddd8cf967c2cb5be99434ad6d6b5192b5660b0
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built sacremoses PyYAML
ERROR: korpora 0.1.1 has requirement 

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
from ratsnlp import nlpbook
args = nlpbook.TrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_root_dir="/root/Korpora",
    downstream_corpus_name="nsmc",
    downstream_task_name="document-classification",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-cls",
    do_eval=True,
    batch_size=32,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [4]:
nlpbook.seed_setting(args)

# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [5]:
nlpbook.set_logger(args)

09/30/2020 07:43:16 - INFO - ratsnlp.nlpbook.utils -   Training/evaluation parameters TrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/root/Korpora', downstream_task_name='document-classification', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-cls', max_seq_length=128, overwrite_model=False, save_top_k=1, monitor='max val_acc', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-06, optimizer='AdamW', lr_scheduler='exp', epochs=20, batch_size=32, cpu_workers=2, fp16=False, do_train=True, do_eval=True, do_predict=False, tpu_cores=0, report_cycle=100, stat_window_length=30)


# 말뭉치 다운로드
실습에 사용할 말뭉치(Naver Sentiment Movie review Corpus)를 다운로드합니다.

In [6]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 48.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 35.0MB/s]                            


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

09/30/2020 07:43:16 - INFO - transformers.tokenization_utils_base -   Model name 'beomi/kcbert-base' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'beomi/kcbert-base' is a path, a model identifier, or url to a directory containing tokenizer files.
09/30/2020 07:43:16 - INFO - filelock -   Lock 139819002641880 acquired on /root/.cache/torch/transformers/8d1e655d205732689406462e2fa1fa62566629a0625aa980eeae4599d873bb66.4e15945a0369

09/30/2020 07:43:17 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/vocab.txt in cache at /root/.cache/torch/transformers/8d1e655d205732689406462e2fa1fa62566629a0625aa980eeae4599d873bb66.4e15945a03694aa613f931134ea9a6f64624cd748a3cfe607ca1e2b066eb9f91
09/30/2020 07:43:17 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/8d1e655d205732689406462e2fa1fa62566629a0625aa980eeae4599d873bb66.4e15945a03694aa613f931134ea9a6f64624cd748a3cfe607ca1e2b066eb9f91
09/30/2020 07:43:17 - INFO - filelock -   Lock 139819002641880 released on /root/.cache/torch/transformers/8d1e655d205732689406462e2fa1fa62566629a0625aa980eeae4599d873bb66.4e15945a03694aa613f931134ea9a6f64624cd748a3cfe607ca1e2b066eb9f91.lock


09/30/2020 07:43:18 - INFO - filelock -   Lock 139819002641880 acquired on /root/.cache/torch/transformers/22b5f7b39de8c16e82d058e2d5116222ce1fc616a291c5a6ad9c2c24e802104f.53b84dc0c694dad783dde1213af7f7c990c093d7453b07066497d5ffcc953289.lock
09/30/2020 07:43:18 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpygjhzlu_


09/30/2020 07:43:19 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/tokenizer_config.json in cache at /root/.cache/torch/transformers/22b5f7b39de8c16e82d058e2d5116222ce1fc616a291c5a6ad9c2c24e802104f.53b84dc0c694dad783dde1213af7f7c990c093d7453b07066497d5ffcc953289
09/30/2020 07:43:19 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/22b5f7b39de8c16e82d058e2d5116222ce1fc616a291c5a6ad9c2c24e802104f.53b84dc0c694dad783dde1213af7f7c990c093d7453b07066497d5ffcc953289
09/30/2020 07:43:19 - INFO - filelock -   Lock 139819002641880 released on /root/.cache/torch/transformers/22b5f7b39de8c16e82d058e2d5116222ce1fc616a291c5a6ad9c2c24e802104f.53b84dc0c694dad783dde1213af7f7c990c093d7453b07066497d5ffcc953289.lock


09/30/2020 07:43:19 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/vocab.txt from cache at /root/.cache/torch/transformers/8d1e655d205732689406462e2fa1fa62566629a0625aa980eeae4599d873bb66.4e15945a03694aa613f931134ea9a6f64624cd748a3cfe607ca1e2b066eb9f91
09/30/2020 07:43:19 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/added_tokens.json from cache at None
09/30/2020 07:43:19 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/special_tokens_map.json from cache at None
09/30/2020 07:43:19 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/tokenizer_config.json from cache at /root/.cache/torch/transformers/22b5f7b39de8c16e82d058e2d5116222ce1fc616a291c5a6ad9c2c

# 학습데이터 구축
학습데이터를 만듭니다.

In [8]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

09/30/2020 07:43:19 - INFO - filelock -   Lock 139818989014544 acquired on /root/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification.lock
09/30/2020 07:43:19 - INFO - ratsnlp.nlpbook.classification.corpus -   Creating features from dataset file at /root/Korpora/nsmc/ratings_train.txt
09/30/2020 07:43:19 - INFO - ratsnlp.nlpbook.classification.corpus -   loading train data... LOOKING AT /root/Korpora/nsmc/ratings_train.txt
09/30/2020 07:43:20 - INFO - ratsnlp.nlpbook.classification.corpus -   tokenize sentences, it could take a lot of time...
09/30/2020 07:43:51 - INFO - ratsnlp.nlpbook.classification.corpus -   tokenize sentences [took 30.568 s]
09/30/2020 07:43:52 - INFO - ratsnlp.nlpbook.classification.corpus -   *** Example ***
09/30/2020 07:43:52 - INFO - ratsnlp.nlpbook.classification.corpus -   sentence: 아 더빙.. 진짜 짜증나네요 목소리
09/30/2020 07:43:52 - INFO - ratsnlp.nlpbook.classification.corpus -   tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] 

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [9]:
if args.do_eval:
    val_dataset = ClassificationDataset(
        args=args,
        corpus=corpus,
        tokenizer=tokenizer,
        mode="test",
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        sampler=SequentialSampler(val_dataset),
        collate_fn=nlpbook.data_collator,
        drop_last=False,
        num_workers=args.cpu_workers,
    )
else:
    val_dataloader = None

09/30/2020 07:44:13 - INFO - filelock -   Lock 139818972797640 acquired on /root/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification.lock
09/30/2020 07:44:13 - INFO - ratsnlp.nlpbook.classification.corpus -   Creating features from dataset file at /root/Korpora/nsmc/ratings_test.txt
09/30/2020 07:44:13 - INFO - ratsnlp.nlpbook.classification.corpus -   loading test data... LOOKING AT /root/Korpora/nsmc/ratings_test.txt
09/30/2020 07:44:14 - INFO - ratsnlp.nlpbook.classification.corpus -   tokenize sentences, it could take a lot of time...
09/30/2020 07:44:25 - INFO - ratsnlp.nlpbook.classification.corpus -   tokenize sentences [took 10.647 s]
09/30/2020 07:44:25 - INFO - ratsnlp.nlpbook.classification.corpus -   *** Example ***
09/30/2020 07:44:25 - INFO - ratsnlp.nlpbook.classification.corpus -   sentence: 굳 ㅋ
09/30/2020 07:44:25 - INFO - ratsnlp.nlpbook.classification.corpus -   tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화합니다.

In [10]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

09/30/2020 07:44:32 - INFO - filelock -   Lock 139818976331816 acquired on /root/.cache/torch/transformers/11ab69ed90bcc2d01ac229deb193678b3b22dc986959a7115be9f7e328d57956.5c73fbc761bca6713f2361fb4816b98c9db40d7c41a6e56197122ef2450ba4b2.lock
09/30/2020 07:44:32 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp5tolfzx5


09/30/2020 07:44:33 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/config.json in cache at /root/.cache/torch/transformers/11ab69ed90bcc2d01ac229deb193678b3b22dc986959a7115be9f7e328d57956.5c73fbc761bca6713f2361fb4816b98c9db40d7c41a6e56197122ef2450ba4b2
09/30/2020 07:44:33 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/11ab69ed90bcc2d01ac229deb193678b3b22dc986959a7115be9f7e328d57956.5c73fbc761bca6713f2361fb4816b98c9db40d7c41a6e56197122ef2450ba4b2
09/30/2020 07:44:33 - INFO - filelock -   Lock 139818976331816 released on /root/.cache/torch/transformers/11ab69ed90bcc2d01ac229deb193678b3b22dc986959a7115be9f7e328d57956.5c73fbc761bca6713f2361fb4816b98c9db40d7c41a6e56197122ef2450ba4b2.lock
09/30/2020 07:44:33 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/beomi/kcbert-base/config.json from cache at /r

In [11]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

09/30/2020 07:44:33 - INFO - filelock -   Lock 139818939870344 acquired on /root/.cache/torch/transformers/a0348cdf9a93056f4e4adc497208b8853967239b4c6acccffcac0196ae7b6c90.3eb9d0c1847ce30bbb6bde7ce4902413737fcd46a212efb3fe8c2a708f2a47d5.lock
09/30/2020 07:44:33 - INFO - transformers.file_utils -   https://cdn.huggingface.co/beomi/kcbert-base/pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp80m2t1qu


09/30/2020 07:44:42 - INFO - transformers.file_utils -   storing https://cdn.huggingface.co/beomi/kcbert-base/pytorch_model.bin in cache at /root/.cache/torch/transformers/a0348cdf9a93056f4e4adc497208b8853967239b4c6acccffcac0196ae7b6c90.3eb9d0c1847ce30bbb6bde7ce4902413737fcd46a212efb3fe8c2a708f2a47d5
09/30/2020 07:44:42 - INFO - transformers.file_utils -   creating metadata file for /root/.cache/torch/transformers/a0348cdf9a93056f4e4adc497208b8853967239b4c6acccffcac0196ae7b6c90.3eb9d0c1847ce30bbb6bde7ce4902413737fcd46a212efb3fe8c2a708f2a47d5
09/30/2020 07:44:42 - INFO - filelock -   Lock 139818939870344 released on /root/.cache/torch/transformers/a0348cdf9a93056f4e4adc497208b8853967239b4c6acccffcac0196ae7b6c90.3eb9d0c1847ce30bbb6bde7ce4902413737fcd46a212efb3fe8c2a708f2a47d5.lock
09/30/2020 07:44:42 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/beomi/kcbert-base/pytorch_model.bin from cache at /root/.cache/torch/transformers/a0348cdf9a93056f4e4

09/30/2020 07:44:47 - WARNING - transformers.modeling_utils -   Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

# 학습 준비
Task와 Trainer를 준비합니다.

In [12]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [13]:
_, trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-cls exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
09/30/2020 07:44:47 - INFO - lightning -   GPU available: True, used: True
TPU available: False, using: 0 TPU cores
09/30/2020 07:44:47 - INFO - lightning -   TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
09/30/2020 07:44:47 - INFO - lightning -   CUDA_VISIBLE_DEVICES: [0]


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-cls`)에 저장됩니다.

In [ ]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)


  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
09/30/2020 07:44:59 - INFO - lightning -   
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
